In [ ]:
import torch
import torch.nn as nn
from torch.distributions import constraints

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO, TraceTMC_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.ops.indexing import Vindex
from pyro.optim import Adam

In [ ]:
data = poly.load_data(poly.JSB_CHORALES)

In [ ]:
seqs = data['train']['sequences']
lengths = data['train']['sequence_lengths']

In [ ]:
seqs.shape

In [ ]:
num_seqs, max_len, data_dim = map(int, seqs.shape)
assert lengths.shape == (num_seqs,)
assert lengths.max() == max_len

In [ ]:
print(num_seqs, max_len, data_dim)

In [ ]:
hd = 4

In [ ]:
probs_x = pyro.sample("probs_x", dist.Dirichlet(0.9 * torch.eye(hd) + 0.1).to_event(1))

In [ ]:
probs_x.shape

In [ ]:
probs_y = pyro.sample(
            "probs_y",
            dist.Beta(0.1, 0.9).expand([hd, 2, data_dim]).to_event(3),
        )

In [ ]:
probs_y.shape

In [ ]:
batch_size = 10

In [ ]:
tones_plate = pyro.plate("tones", data_dim, dim=-1)

In [ ]:
globals().keys()

In [ ]:
with pyro.plate("sequences", num_seqs, batch_size, dim=-2) as batch:
    print(batch)
    lengths = lengths[batch]
    x, y = 0, 0
    print(lengths)
    for t in pyro.markov(range(lengths.max())):
        print(t)